In [ ]:
%matplotlib inline

## Dataset과 DataLoader

데이터 샘플을 처리하는 코드는 지저분 , 유지보수 어려울 수 있다.

더나은 가독성 & 모듈성을 위해 데이터 셋 코드를 학습 코드로부터 분리 하는게 이상적이다.



## 데이터셋 불러오기

Fashion MNIST 데이터셋 불러오기
기사 이미지 셋 training 60,000 test 10,000

각 예제
-흑백(grayscale)의 28x28 이미지
-10개 class 중 하나인 정답 (라밸)

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

데이터를 불러오며 경로지정 (root) , 데이터 종류 결정 (train),


다운로드 여부 (download), 변형방법 지정 (transform) 진행

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

## 데이터셋 순회하고 시각화하기

데이터셋에 리스트 처럼 접근이 가능

matplotlib 사용해서 시각화

In [ ]:
labels_map = {    # 숫자 & 클래스 레이블 - 딕셔너리
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(8, 8)) # 그래프 사이즈
cols , rows = 3, 3 # 행 열
for i in range(1, cols * rows + 1): # 인덱스 번호넣기
    sample_idx = torch.randint(len(training_data), size=(1,)).item() # 무작위로 샘플 인덱스 설정
    img, label = training_data[sample_idx] # 샘플 인덱스에서 이미지 레이블 가져오기
    figure.add_subplot(rows, cols, i) # 서브플롯 집어넣기
    plt.title(labels_map[label]) # 플롯 이름 넣기
    plt.axis("off") # 축 제거
    plt.imshow(img.squeeze(), cmap="gray") # 이미지 변형
plt.show( )

##파일에서 사용자정의 데이터셋 만들기

dataset 클래스는 반드시 3개 함수 구현해야 한다.

__init__,

__len__,

__getitem__

구현을 살피면 이미지는 img_dir 디렉토리에 저장되고,
정답은 annotation_file csv 파일에 별도로 저장된다.

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform = None, target_transform = None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:   # 만약 해당 변환이 지정되어 있다면 적용하는 코드
            image = self.tranform(image)
        if self.target_transform: # 마찬가지
            label = self.target_transform(label)
        return image, label    # 아래 코드와 다르게 이미지 라벨 따로 반환

##init

init 함수는 dataset 객체가 생설될 때 한번씩만 실행

이미지 & 주석파일 (annotation_file) 포함된 디렉터토리

두가지 변형 (transformation) 초기화

In [ ]:
def __init__(self, annotations_file, img_dir, transform = None, target_transform = None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

##len

len 함수는 데이터셋의 샘플 개수를 반환

In [ ]:
def __len__(self):
    return len(self.img_labels)

##getitem

주어진 인덱스의 idx에 해당하는 샘플을 데이터셋에서 불러오고 반환합니다.

인덱스를 기반으로 디스크에서 위치 식별,
read_image 를 사용하여 이미지를 텐서로 변환하고,


self.img_labels 의 csv 데이터로부터 해당하는 정답(label)을 가져오기


해당하는 경우 변형(transform) 함수들을 호출한 뒤, 텐서 이미지와 라벨을 Python 딕셔너리형으로 반환합니다.

In [ ]:
def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = read_image(img_path)
    label = self.img_labels.iloc[idx, 1]
    if self.transform:
        image = self.tranform(image)
    if self.target_transform:
        label = self.target_transform(label)
    sample = {"image": image, "label": label}
    return sample

##DataLoader 로 학습용 데이터 준비하기

dataset 은 특징을 가져오고 하나의 샘플에 label 지정

모델 학습 시, 샘플들을 미니배치로 전달, 매 에폭마다 데이터를 섞어 과적합을 막음

multiprocessing 으로 데이터 검색 속도 높임


DataLoader 은 간단한 API로 이러한 복잡한 과정들을 추상화한 객체

In [ ]:
from torch.utils.data import DataLoader # 라이브러리에 있음!

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

## DataLoader 를 통해 순회하기

DataLoader 에 데이터셋을 부른 뒤

각 순회(iteration)는
(각각 batch_size=64 의 특징(feature)과 정답(label)을 포함하는)

train_features 와 train_labels 의 묶음(batch)을 반환


shuffle=True 로 지정했으므로, 모든 배치를 순회한 뒤 데이터가 섞입니다.

In [ ]:
train_features, train_labels = next(iter(train_dataloader))  # 다음 미니배치를 가져오는 것
print(f"Feature batch shape: {train_features.size()}")  # 미니배치 크기 (shape)
print(f"Labels batch shape: {train_labels.size()}")  # 레이블 미니배치 크기 (shape)
img = train_features[0].squeeze() # 첫번째 이미지, 스퀴즈로 (1, H, W) -> (H, W)
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")